# Check for price outliers

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 0: Create instrument property definitions

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- In this query, we make properties available to Luminesce
-- by writing them to the Lusid.Property.Definition provider.
-- See the following page for further details:
-- https://support.lusid.com/knowledgebase/article/KA-01702/en-us
-- ===============================================================

-- 1. Define new properties

@newProperties =
values
   ('Instrument/ibor/Sector', 'Text', 'Sector', 'The sector that the instrument belongs to.');

@property_definition =
select Column3 as [DisplayName], 'Instrument' as [Domain], 'ibor' as [PropertyScope], Column3 as [PropertyCode], 'Property' as
   [ConstraintStyle], 'system' as [DataTypeScope], case
      when Column2 == 'Text'
         then 'string'
      else 'number'
      end as [DataTypeCode]
from @newProperties;

-- 2. Write new properties to Lusid.Property.Definition provider

@create_properties =
select *
from Lusid.Property.Definition.Writer
where ToWrite = @property_definition;

-- 3. The results of writing the new property definitions can be seen from the query below:

select *
from @create_properties;



#### Step 1: Setup instruments with properties

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- In this query we setup some equity instruments
-- NOTE: You'll need to have "Sector" setup as an instrument
-- property in LUSID and Luminesce as follows:
-- Instrument/ibor/Sector
-- ============================================================
-- Load data from CSV
@instruments_data =

use Drive.Excel
--file=/luminesce-examples/price_time_series.xlsx
--worksheet=instrument
enduse;

-- 1. Upload values for custom instrument properties
-- Transform data
@ids =
select inst_id as Id
from @instruments_data;

@inst_properties =
select li.LusidInstrumentId as EntityId, 'LusidInstrumentId' as EntityIdType, 'Instrument' as Domain, 'ibor' as PropertyScope, a.PropertyCode, a.
   Value
from Lusid.Instrument li
inner join (
   select 'Sector' as PropertyCode, sector as Value, inst_id as EntityId
   from @instruments_data
   ) a
   on li.ClientInternal = a.EntityId
where li.ClientInternal in @ids;

-- Write data to Lusid.Property
select *
from Lusid.Property.Writer
where ToWrite = @inst_properties;

-- 2. Upload instrument equity data to inbuilt properties
-- Transform equity data
@equity_instruments =
select inst_id as ClientInternal, name as DisplayName, ccy as InferredDomCcy
from @instruments_data;

-- Write data to Lusid.Instrument.Equity. Print results of writing data to console.
select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @equity_instruments
   and DeletePropertiesWhereNull = True;



#### Step 2: Upload quotes

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- In this query we load a time series of equity prices
-- Some of the prices will have outlier values
-- ============================================================
-- 1. Load data from CSV
@quotes_data = use Drive.Excel
--file=/luminesce-examples/price_time_series.xlsx
--worksheet=price_time_series
enduse;

-- 2. Transform quotes data
@quotes_for_upload =
    select
        'ClientInternal' as InstrumentIdType,
        instrument_id as Instrumentid,
        'luminesce-examples' as QuoteScope,
        'Price' as QuoteType,
        'Lusid' as Provider,
        'Mid' as Field,
        price_date as QuoteEffectiveAt,
        price as Value,
        ccy as Unit
    from @quotes_data;

-- 3. Upload quotes into LUSID. Print results of upload to console.
select *
from Lusid.Instrument.Quote.Writer
where ToWrite = @quotes_for_upload;


#### Step 3: Create iqr checker view

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- Here we build a view which will return outliers for a given
-- instrument and date range. We define outliers as observations
-- that fall below Q1 - 1.5 IQR or above Q3 + 1.5 IQR
-- ============================================================
-- 1. Create view and set parameters
@outlier_view = use Sys.Admin.SetupView
--provider=Custom.PriceCheck.OnePointFiveIQR
--parameters
StartDate,Date,2022-01-01,true
EndDate,Date,2022-12-31,true
InstId,Text,EQ56JD720345,true
----

@@StartDate = select #PARAMETERVALUE(StartDate);
@@EndDate = select #PARAMETERVALUE(EndDate);
@@InstId = select #PARAMETERVALUE(InstId);

-- 2. Collect quotes for instrument

@quotes_data = select *
    from Lusid.Instrument.Quote
    where QuoteScope='luminesce-examples'
        and InstrumentIdType = 'ClientInternal'
        and QuoteType = 'Price'
        and InstrumentId = @@InstId
        and QuoteEffectiveAt between @@StartDate and @@EndDate;

-- 3. Collect instrument static and join on data for sector instrument property

@instrument_data = select
    i.ClientInternal,
    i.DisplayName,
    p.Value as Sector,
    i.InferredAssetClass as [AssetClass]
    from Lusid.Instrument.Equity i
    join Lusid.Instrument.Property p
        on p.InstrumentId=i.LusidInstrumentId
    where i.ClientInternal = @@InstId
        and propertyscope = 'ibor'
        and propertycode = 'Sector';

-- 4. Generate time series

@price_ts = select
    ClientInternal,
    DisplayName,
    Sector,
    AssetClass,
    QuoteEffectiveAt as [PriceDate],
    Unit as [Currency],
    Value as [Price]
    from @instrument_data i
    join @quotes_data q on (i.ClientInternal = q.InstrumentId);

-- 5. Run IQR checks

@iqr_data = select
    interquartile_range(price) * (1.5) as [iqr_x1_5],
    quantile(price, 0.25) as [q1],
    quantile(price, 0.75) as [q3]
    from @price_ts;

-- 6. Define and upper and lower limit for our price check

@@upper_limit = select (q3 + iqr_x1_5 ) from  @iqr_data;
@@lower_limit = select (q1 - iqr_x1_5 ) from  @iqr_data;

-- 7. Print upper and lower limits to console

@@upper_limit_log = select print('Upper limit for outlier check: {X:00000} ', '', 'Logs', @@upper_limit);
@@lower_limit_log = select print('Lower limit for outlier check: {X:00000} ', '', 'Logs', @@lower_limit);

select
    PriceDate,
    ClientInternal,
    DisplayName,
    @@upper_limit as [UpperLimit],
    @@lower_limit as [LowerLimit],
    Price,
    'Outlier' as Result
from @price_ts
where price not between @@lower_limit and @@upper_limit;

enduse;

select * from @outlier_view ;


#### Step 4: Create price outlier view

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- Here we build a view which will return outlier for all
-- Equities in a given sector between two date ranges
-- ============================================================
-- 1. Create view and set parameters
@price_check_view =

use Sys.Admin.SetupView
--provider=DataQc.OutlierCheck.Prices
--parameters
StartDate,Date,2022-01-01,true
EndDate,Date,2022-12-31,true
Sector,Text,Technology,true
AssetClass,Text,Equity,true
----

@@Sector = select #PARAMETERVALUE(Sector);
@@StartDate = select #PARAMETERVALUE(StartDate);
@@EndDate = select #PARAMETERVALUE(EndDate);
@@AssetClass = select #PARAMETERVALUE(AssetClass);

-- 2. Collect quotes for instrument

@instrument_data =
    select
        i.ClientInternal
    from Lusid.Instrument.Property p
    join Lusid.Instrument.Equity i
        on p.InstrumentId = i.LusidInstrumentId
    where i.[Type]=@@AssetClass
        and p.propertyscope = 'ibor'
        and p.propertycode = 'Sector'
        and p.value = @@Sector;

-- 3. Collect instrument static and print view of the outliers for the given sector & date range to console

select
    i.ClientInternal,
    r.DisplayName,
    r.PriceDate,
    r.Price,
    r.LowerLimit,
    r.UpperLimit,
    r.Result
from
    @instrument_data i
    cross apply
    (
        select * from
        Custom.PriceCheck.OnePointFiveIQR iqr
        where iqr.StartDate = @@StartDate
        and iqr.EndDate = @@EndDate
    ) r
    where r.ClientInternal = i.ClientInternal

enduse;

select *
from @price_check_view;


#### Step 5: Run price outlier view

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- This is the high level query which an end-user might run
-- to collect all outlier prices between two dates for
-- a Sector
-- ============================================================

select *
from DataQc.OutlierCheck.Prices
where Sector = 'Technology'
and AssetClass= 'Equity'
and StartDate = #2022-01-01#
and EndDate = #2022-08-26#

